In [ ]:
from deep_orderbook.shaper import BookShaper, ArrayShaper
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn_image as isns
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
MARKETS = ["ETH-BTC", "BTC-USD", "ETH-USD"]

In [ ]:
# itertools for asyncio !
import asyncio
from aioitertools import iter, next, map, zip
from aioitertools import enumerate

In [ ]:
from deep_orderbook.feeds.coinbase_feed import CoinbaseFeed
from deep_orderbook.replayer import ParquetReplayer

async def iter_sec(date_regexp='2024-08-06', max_samples=0):
    replayer = ParquetReplayer('data', date_regexp=date_regexp)
    async with CoinbaseFeed(
        markets=MARKETS,
        replayer=replayer,
    ) as feed:
        async for onesec in feed.one_second_iterator():
            yield onesec
            if max_samples == 0:
                break
            else:
                max_samples -= 1

async for onesec in iter_sec(max_samples=3):
    print(onesec)

In [ ]:
async def iter_shapes(date_regexp='2024-08-06', max_samples=0):
    symbol_shapers = {pair: ArrayShaper(zoom_frac=0.01) for pair in MARKETS}
    async for onesec in iter_sec(max_samples=max_samples):
        yield {
            market: await symbol_shapers[market].make_arr3d(
                onesec.symbols[market]
            )
            for market in MARKETS
        }
        if max_samples == 0:
            break
        else:
            max_samples -= 1


async for shaped in iter_shapes(max_samples=1):
    print(shaped)


In [ ]:
async def iter_gen_arrays(date_regexp='2024-08-06', max_samples=0):
    async for shapes in iter_shapes(max_samples=max_samples):
        yield shapes['BTC-USD'].transpose(1, 0, 2)
#        yield np.hstack([shapes[market] for market in MARKETS]).transpose(1, 0, 2)


plt.figure(figsize=(25,11))
async for arr in iter_gen_arrays(max_samples=1000):
    im = arr.copy()
    im[:,:,0] /= 10
    # im[:,:,1] *= 10
    # im[:,:,2] *= 10
    im += 0.5
    im = np.roll(im, 2, axis=2)
    print(im.min(), im.max())
    im = im.clip(0, 1)
    isns.imshow(im, origin='lower', cmap='inferno')#, gray=True)
    clear_output(wait=True)
    plt.show()
    print(arr.shape)
    # im[:,:,1] *= 10
    # im[:,:,2] *= 10


In [ ]:
every = 2
LENGTH = 256
x = []
async for n,sec in enumerate(genacc):
    allim = []
    for symb, data in sec.items():
        arr = np.stack(data['bs'][-LENGTH:])
        im = arr
        im[:,:,0] /= 10
        im += 0.5
        allim.append(im)
    allim = np.concatenate(allim, axis=1)
    if n % every == 0:
        plt.figure(figsize=(15,11))
        toshow = allim.transpose(1,0,2)
        toshow = np.clip(toshow, 0, 1)
        isns.imshow(toshow, origin="lower")
        plt.show()
        # pr = np.stack(data['ps'])[-LENGTH:, :-1]
        # plt.plot(pr)
        # plt.show()
#        print(symb)
        clear_output(wait=True)
#        plt.show()
#    if n == 100:
#        break
    pass